In [1]:
import pandas as pd

In [2]:
import numpy as np

### Carregar dados

In [3]:
jogos = pd.read_csv("data/brasileiro-a.csv")

In [4]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2017,30,23.10. 19:00,Botafogo,Corinthians,2 : 1
1,brasileiro-a,2017,30,22.10. 18:00,Chapecoense,Fluminense,2 : 0
2,brasileiro-a,2017,30,22.10. 18:00,Ponte Preta,Avaí,1 : 2
3,brasileiro-a,2017,30,22.10. 16:00,Atlético-PR,Sport Recife,2 : 1
4,brasileiro-a,2017,30,22.10. 16:00,Bahia,Vitória,2 : 1


#### A query abaixo possibilita restringir o espaço amostral

In [5]:
#jogos.query('ano == 2017 & rodada > 25', inplace=True)

In [6]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2017,30,23.10. 19:00,Botafogo,Corinthians,2 : 1
1,brasileiro-a,2017,30,22.10. 18:00,Chapecoense,Fluminense,2 : 0
2,brasileiro-a,2017,30,22.10. 18:00,Ponte Preta,Avaí,1 : 2
3,brasileiro-a,2017,30,22.10. 16:00,Atlético-PR,Sport Recife,2 : 1
4,brasileiro-a,2017,30,22.10. 16:00,Bahia,Vitória,2 : 1


In [7]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6046 entries, 0 to 6045
Data columns (total 7 columns):
campeonato    6046 non-null object
ano           6046 non-null int64
rodada        6046 non-null int64
data          6046 non-null object
mandante      6046 non-null object
visitante     6046 non-null object
placar        6046 non-null object
dtypes: int64(2), object(5)
memory usage: 330.7+ KB


In [8]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [9]:
times = list(jogos["mandante"].unique())

In [10]:
times.sort()

In [11]:
times

['América-MG',
 'América-RN',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Barueri',
 'Botafogo',
 'Brasiliense',
 'Ceará',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Fortaleza',
 'Goias',
 'Grêmio',
 'Guarani',
 'Internacional',
 'Ipatinga',
 'Joinville',
 'Juventude',
 'Náutico',
 'Palmeiras',
 'Paraná',
 'Paysandu',
 'Ponte Preta',
 'Portuguesa',
 'Santa Cruz',
 'Santo Andre',
 'Santos',
 'Sao Caetano',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [12]:
len(times)

41

### Transformar placar em 2 colunas

In [13]:
def tratarPlacar(row, mandante):
    placar = row["placar"].split(" : ")
    if (mandante):
        return int(placar[0])
    return int(placar[1])

In [14]:
jogos['gols_mandante'] = jogos.apply (lambda row: tratarPlacar(row, True),axis=1)

In [15]:
jogos['gols_visitante'] = jogos.apply (lambda row: tratarPlacar(row, False),axis=1)

In [16]:
del jogos["placar"]

In [17]:
jogos.describe()

,ano,rodada,gols_mandante,gols_visitante
count,6046.000000,6046.000000,6046.000000,6046.000000
mean,2009.469732,20.184750,1.601886,1.050612
std,4.407865,11.568851,1.256693,1.043830
min,2003.000000,1.000000,0.000000,0.000000
25%,2005.000000,10.000000,1.000000,0.000000
50%,2009.000000,20.000000,1.000000,1.000000
75%,2013.000000,30.000000,2.000000,2.000000
max,2017.000000,46.000000,7.000000,7.000000


### Tratar coluna de data

In [18]:
def tratarData(row):
    data = row["data"].split(".")    
    return data[0] + "-" + data[1]

In [19]:
jogos['data'] = jogos.apply (lambda row: tratarData(row),axis=1)

In [20]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante
0,brasileiro-a,2017,30,23-10,Botafogo,Corinthians,2,1
1,brasileiro-a,2017,30,22-10,Chapecoense,Fluminense,2,0


### Criar coluna com resultado

In [21]:
def resultado(row):
    if (row["gols_mandante"] > row["gols_visitante"]):
        return "MANDANTE"
    elif (row["gols_mandante"] < row["gols_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [22]:
jogos['resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [23]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2017,30,23-10,Botafogo,Corinthians,2,1,MANDANTE
1,brasileiro-a,2017,30,22-10,Chapecoense,Fluminense,2,0,MANDANTE
2,brasileiro-a,2017,30,22-10,Ponte Preta,Avaí,1,2,VISITANTE
3,brasileiro-a,2017,30,22-10,Atlético-PR,Sport Recife,2,1,MANDANTE
4,brasileiro-a,2017,30,22-10,Bahia,Vitória,2,1,MANDANTE


In [24]:
jogos.to_csv("data/jogos-resultado.csv", sep=",")

### Totais de resultados por ano

In [25]:
jogos.groupby(["ano", "resultado"])[["ano"]].count()

ano
ano  resultado     
2003 EMPATE     142
     MANDANTE   297
     VISITANTE  113
2004 EMPATE     140
     MANDANTE   288
     VISITANTE  124
2005 EMPATE     103
     MANDANTE   235
     VISITANTE  124
2006 EMPATE      97
     MANDANTE   191
     VISITANTE   92
2007 EMPATE      90
     MANDANTE   192
     VISITANTE   98
2008 EMPATE      96
     MANDANTE   208
     VISITANTE   76
2009 EMPATE     102
     MANDANTE   195
     VISITANTE   83
2010 EMPATE     118
     MANDANTE   179
     VISITANTE   83
2011 EMPATE     105
     MANDANTE   184
     VISITANTE   91
2012 EMPATE     105
     MANDANTE   183
     VISITANTE   92
2013 EMPATE     108
     MANDANTE   184
     VISITANTE   88
2014 EMPATE      92
     MANDANTE   197
     VISITANTE   91
2015 EMPATE      91
     MANDANTE   200
     VISITANTE   89
2016 EMPATE      95
     MANDANTE   202
     VISITANTE   83
2017 EMPATE      77
     MANDANTE   130
     VISITANTE   93

### Totais por resultado

In [26]:
totais_resultados = jogos.groupby("resultado").size()

In [27]:
total_jogos = totais_resultados.sum()

In [28]:
resultados_DF = pd.DataFrame({'resultados': totais_resultados.keys(), 'totais': totais_resultados.values})

In [29]:
resultados_DF

,resultados,totais
0,EMPATE,1561
1,MANDANTE,3065
2,VISITANTE,1420


In [30]:
def percentual(valor, total):
    return round(valor/total, 4)

In [31]:
resultados_DF['%'] = resultados_DF.apply (lambda row: percentual(row["totais"], total_jogos),axis=1)

In [32]:
resultados_DF

,resultados,totais,%
0,EMPATE,1561,0.2582
1,MANDANTE,3065,0.5069
2,VISITANTE,1420,0.2349


### Calcular resultados de um time

In [33]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["resultado"] == resultado)]) 

In [34]:
resultados_time_DF = pd.DataFrame({'time': times })

In [35]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "MANDANTE"),axis=1)

In [36]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,América-MG,14,10,14,1,10,27
1,América-RN,2,3,14,2,2,15
2,Atlético-GO,24,15,33,13,20,39


In [37]:
resultados_time_DF.eval("jogos_casa = vitorias_casa + empates_casa + derrotas_casa", inplace=True)
resultados_time_DF.eval("jogos_fora = vitorias_fora + empates_fora + derrotas_fora", inplace=True)
resultados_time_DF.eval("jogos_total = jogos_fora + jogos_fora", inplace=True)

In [38]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total
0,América-MG,14,10,14,1,10,27,38,38,76
1,América-RN,2,3,14,2,2,15,19,19,38
2,Atlético-GO,24,15,33,13,20,39,72,72,144


In [39]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [40]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa
0,América-MG,14,10,14,1,10,27,38,38,76,0.456140
1,América-RN,2,3,14,2,2,15,19,19,38,0.157895
2,Atlético-GO,24,15,33,13,20,39,72,72,144,0.402778


In [41]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [42]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora
0,América-MG,14,10,14,1,10,27,38,38,76,0.456140,0.114035
1,América-RN,2,3,14,2,2,15,19,19,38,0.157895,0.140351
2,Atlético-GO,24,15,33,13,20,39,72,72,144,0.402778,0.273148


In [43]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [44]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total
38,São Paulo,175,69,47,98,76,117,291,291,582,0.680412,0.423826,0.552119
12,Corinthians,153,67,52,86,87,99,272,272,544,0.644608,0.422794,0.533701
15,Cruzeiro,166,66,59,104,55,132,291,291,582,0.646048,0.420389,0.533219


In [45]:
resultados_time_DF = resultados_time_DF[["time", "jogos_casa", "jogos_fora", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

In [46]:
def golsMandante(time):
    return jogos[jogos["mandante"] == time][["gols_mandante"]].sum()

In [47]:
def golsVisitante(time):
    return jogos[jogos["visitante"] == time]["gols_visitante"].sum()

In [48]:
resultados_time_DF["gols_mandante"] = resultados_time_DF.apply (lambda row: golsMandante(row["time"]),axis=1)
resultados_time_DF["gols_visitante"] = resultados_time_DF.apply (lambda row: golsVisitante(row["time"]),axis=1)

resultados_time_DF.eval("total_gols = gols_mandante + gols_visitante", inplace=True)
resultados_time_DF.eval("media_gols_casa = gols_mandante/jogos_casa", inplace=True)
resultados_time_DF.eval("media_gols_fora = gols_visitante/jogos_fora", inplace=True)
resultados_time_DF.eval("media_gols = (gols_mandante + gols_visitante)/jogos_total", inplace=True)

In [49]:
resultados_time_DF.head()


,time,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
38,São Paulo,291,291,582,0.680412,0.423826,0.552119,524,360,884,1.800687,1.237113,1.518900
12,Corinthians,272,272,544,0.644608,0.422794,0.533701,428,309,737,1.573529,1.136029,1.354779
15,Cruzeiro,291,291,582,0.646048,0.420389,0.533219,546,360,906,1.876289,1.237113,1.556701
35,Santos,291,291,582,0.674685,0.366552,0.520619,540,352,892,1.855670,1.209622,1.532646
23,Internacional,276,276,552,0.661836,0.361111,0.511473,442,296,738,1.601449,1.072464,1.336957


In [50]:
resultados_time_DF = resultados_time_DF[["time", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total", "media_gols_casa", "media_gols_fora", "media_gols"]]

In [51]:
resultados_time_DF.set_index("time", inplace=True)

In [52]:
resultados_time_DF.sort_values(["aprov_total"], ascending = [0]).head(3)

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
time,,,,,,,
São Paulo,582,0.680412,0.423826,0.552119,1.800687,1.237113,1.518900
Corinthians,544,0.644608,0.422794,0.533701,1.573529,1.136029,1.354779
Cruzeiro,582,0.646048,0.420389,0.533219,1.876289,1.237113,1.556701


In [53]:
resultados_time_DF_DESC = resultados_time_DF.describe(percentiles=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

In [54]:
resultados_time_DF_DESC

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,294.926829,0.547364,0.270184,0.408774,1.502899,0.963321,1.233110
std,204.188196,0.096643,0.089615,0.084039,0.248162,0.188657,0.194113
min,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
0%,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
10%,42.000000,0.438596,0.140351,0.307018,1.238095,0.746032,1.076923
20%,76.000000,0.500000,0.198413,0.350877,1.315789,0.871622,1.131579
30%,134.000000,0.526316,0.210526,0.376923,1.412429,0.901099,1.166667
40%,172.000000,0.539048,0.262857,0.383041,1.461538,0.947368,1.210046
50%,210.000000,0.555556,0.271062,0.414414,1.547009,1.000000,1.226351


In [55]:
percentis = resultados_time_DF_DESC.loc[["10%","20%","30%","40%","50%","60%","70%","80%","90%"]]

In [56]:
percentis

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
10%,42.0,0.438596,0.140351,0.307018,1.238095,0.746032,1.076923
20%,76.0,0.500000,0.198413,0.350877,1.315789,0.871622,1.131579
30%,134.0,0.526316,0.210526,0.376923,1.412429,0.901099,1.166667
40%,172.0,0.539048,0.262857,0.383041,1.461538,0.947368,1.210046
50%,210.0,0.555556,0.271062,0.414414,1.547009,1.000000,1.226351
60%,354.0,0.580952,0.295238,0.430199,1.578947,1.029412,1.289474
70%,468.0,0.596825,0.329060,0.457364,1.622857,1.057143,1.345382
80%,544.0,0.602941,0.359069,0.481005,1.698529,1.106838,1.391753
90%,582.0,0.658088,0.376861,0.511473,1.776256,1.164659,1.440299


In [57]:
def get_stars(array,value):
    print ()
    idx = (np.abs(array-value)).argmin()
    near = array[idx][0]
    index = np.where(array==near)[0][0]
    
    if (value > near):
        index = index + 1  
       
    return index + 1

In [58]:
stars_DF = resultados_time_DF.copy()

In [59]:
stars_DF["aprov_total"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_total"]].values, row["aprov_total"]),axis=1)

In [60]:
stars_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_casa"]].values, row["aprov_casa"]),axis=1)

In [61]:
stars_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_fora"]].values, row["aprov_fora"]),axis=1)

In [62]:
stars_DF["media_gols_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_casa"]].values, row["media_gols_casa"]),axis=1)

In [63]:
stars_DF["media_gols_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_fora"]].values, row["media_gols_fora"]),axis=1)

In [64]:
stars_DF["media_gols"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols"]].values, row["media_gols"]),axis=1)

In [65]:
stars_DF.eval("forca_casa = (3*aprov_casa + media_gols_casa)/4", inplace=True)

In [66]:
stars_DF.eval("forca_fora = (3*aprov_fora + media_gols_fora)/4", inplace=True)

In [67]:
stars_DF.eval("forca = (3*aprov_total + 1*media_gols)/4", inplace=True)

In [68]:
stars_DF.sort_values(["forca"], ascending=[0])

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols,forca_casa,forca_fora,forca
time,,,,,,,,,,
São Paulo,582,10,10,10,10,10,10,10.00,10.00,10.00
Santos,582,10,9,10,10,10,10,10.00,9.25,10.00
Cruzeiro,582,9,10,10,10,10,10,9.25,10.00,10.00
Corinthians,544,9,10,10,6,9,8,8.25,9.75,9.50
Palmeiras,498,9,10,9,8,9,9,8.75,9.75,9.00
Grêmio,540,10,8,9,9,6,8,9.75,7.50,8.75
Internacional,552,10,9,9,7,8,7,9.25,8.75,8.50
Flamengo,582,8,9,9,6,7,7,7.50,8.50,8.50
Atlético-MG,544,8,8,8,9,9,10,8.25,8.25,8.50


In [67]:
stars_DF.to_csv(path_or_buf="stars.csv",sep=";")